In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/kcc_휴먼이해논문공모전

/content/drive/MyDrive/kcc_휴먼이해논문공모전


In [ ]:
!git clone https://github.com/catSirup/KorEDA.git

In [3]:
import random
import pickle
import re

wordnet = {}
with open("/content/drive/MyDrive/kcc_휴먼이해논문공모전/KorEDA/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

In [4]:
def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
  words=sentence

  augmented_sentences = []
  num_new_per_technique = int(num_aug/4) + 1

  num_words = len(sentence) ##추가

  n_sr = max(1, int(alpha_sr*num_words))
  n_ri = max(1, int(alpha_ri*num_words))
  n_rs = max(1, int(alpha_rs*num_words))

  # sr
  for _ in range(num_new_per_technique):
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences.append(' '.join(a_words))

	# ri
  for _ in range(num_new_per_technique):
    a_words = random_insertion(words, n_ri)
    augmented_sentences.append(' '.join(a_words))

	# rs  
  for _ in range(num_new_per_technique):
    a_words = random_swap(words, n_rs)
    augmented_sentences.append(" ".join(a_words))

	# rd
  for _ in range(num_new_per_technique):
    a_words = random_deletion(words, p_rd)
    augmented_sentences.append(" ".join(a_words))

  augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
  random.shuffle(augmented_sentences)

  if num_aug >= 1:
    augmented_sentences = augmented_sentences[:num_aug]
  else:
    keep_prob = num_aug / len(augmented_sentences)
    augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

  #augmented_sentences.append(sentence)

  return augmented_sentences

test 데이터의 세션 : {'08', '18', '19', '24', '36', '37', '38', '40'}

In [41]:
import pickle
with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/text_processing.pkl',"rb") as fr:
    d = pickle.load(fr)

d

,Segment ID,sentiment,preprocessing
0,Sess01_script01_User002M_001,0,"[나, 는, 어, 어머니, 엄마, 가, 이제, 생일, 때, 마다, 이제, 미역국, ..."
1,Sess01_script01_User002M_002,0,"[전, 에도, 말, 했지만, 그래서, 미역국, 이랑, 볶음, 김치, 막, 해주셔서,..."
2,Sess01_script01_User002M_003,0,"[어, 응, 거기, 를, 되게, 엄, 집, 앞, 에, 그, 조그만, 한, 가게, 로..."
3,Sess01_script01_User002M_004,0,"[거기, 서, 엄마, 랑, 둘, 이, 생일, 파티, 그냥, 간단하게, 하는, 경우,..."
4,Sess01_script01_User001F_001,0,"[아, 친구, 들, 도]"
...,...,...,...
13435,Sess40_script06_User079F_043,0,"[뭔가, 막, 그게, 그니까, 완전, 대중화, 가, 된, 시대, 가, 된, 거야]"
13436,Sess40_script06_User079F_044,0,"[그, 게임, 속, 그, 스토리, 상, 에서는]"
13437,Sess40_script06_User079F_045,0,"[근데, 그게, 너무, 저렴하고, 하니까, 다, 갈아, 치우고, 막, 내, 아내, ..."
13438,Sess40_script06_User079F_046,0,"[왜냐면, 내, 아내, 를, 대신, 할, 사람, 이, 있는, 거야]"


In [42]:
test_list = []
train_list = []
for i in range(13440 ):
  if int(d['Segment ID'][i][4:6]) in [8, 18, 19, 24, 36, 37, 38, 40]:
    test_list.append(i)
  else:
    train_list.append(i) 

print(len(test_list))
print(len(train_list))

2614
10826


In [43]:
train = d.iloc[train_list].reset_index(drop=True)
train

,Segment ID,sentiment,preprocessing
0,Sess01_script01_User002M_001,0,"[나, 는, 어, 어머니, 엄마, 가, 이제, 생일, 때, 마다, 이제, 미역국, ..."
1,Sess01_script01_User002M_002,0,"[전, 에도, 말, 했지만, 그래서, 미역국, 이랑, 볶음, 김치, 막, 해주셔서,..."
2,Sess01_script01_User002M_003,0,"[어, 응, 거기, 를, 되게, 엄, 집, 앞, 에, 그, 조그만, 한, 가게, 로..."
3,Sess01_script01_User002M_004,0,"[거기, 서, 엄마, 랑, 둘, 이, 생일, 파티, 그냥, 간단하게, 하는, 경우,..."
4,Sess01_script01_User001F_001,0,"[아, 친구, 들, 도]"
...,...,...,...
10821,Sess39_script06_User078F_038,0,"[그거, 자체, 가, 꼰대, 같은, 느낌, 도, 들어]"
10822,Sess39_script06_User077F_035,0,"[아무튼, 어, 꽃, 아, 맞아]"
10823,Sess39_script06_User078F_039,2,"[성적, 인, 것, 도, 있지만, 꼰대, 지]"
10824,Sess39_script06_User077F_036,0,"[그, 당연히, 그것, 도, 있지만, 어, 꼰대, 그치]"


In [44]:
test = d.iloc[test_list].reset_index(drop=True)
test

,Segment ID,sentiment,preprocessing
0,Sess08_script01_User015F_001,1,"[나, 난, 근데, 아까, 봤던, 게, 좀, 그거, 봤던, 것, 들, 이라서]"
1,Sess08_script01_User016F_001,0,"[나, 첫, 번째, 것, 만, 봤었어]"
2,Sess08_script01_User015F_002,0,"[근데, 막, 돌아다니는, 거, 많잖아]"
3,Sess08_script01_User016F_002,0,"[맞아, 무한도전, 은]"
4,Sess08_script01_User015F_003,1,"[그, 맨날, 페북, 이랑, 막, 그런데, 그래가지고, 난, 별로, 재미, 가, 없었어]"
...,...,...,...
2609,Sess40_script06_User079F_043,0,"[뭔가, 막, 그게, 그니까, 완전, 대중화, 가, 된, 시대, 가, 된, 거야]"
2610,Sess40_script06_User079F_044,0,"[그, 게임, 속, 그, 스토리, 상, 에서는]"
2611,Sess40_script06_User079F_045,0,"[근데, 그게, 너무, 저렴하고, 하니까, 다, 갈아, 치우고, 막, 내, 아내, ..."
2612,Sess40_script06_User079F_046,0,"[왜냐면, 내, 아내, 를, 대신, 할, 사람, 이, 있는, 거야]"


In [45]:
train['sentiment'].value_counts()

0    9026
1    1152
2     198
3     164
4     156
5      85
6      45
Name: sentiment, dtype: int64

In [46]:
test['sentiment'].value_counts()

0    2094
1     376
3      48
4      36
2      29
5      16
6      15
Name: sentiment, dtype: int64

In [47]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train[['Segment ID','preprocessing']], train['sentiment'], test_size=0.125, shuffle=True, random_state=4)

x_valid

,Segment ID,preprocessing
4710,Sess16_script01_User031M_007,"[그, 캐, 그, 뭐, 뭐, 냐, 잭스, 패, 로우]"
9903,Sess34_script03_User068M_031,"[그치, 공항, 에, 연예인, 많지]"
8327,Sess30_script02_User059F_018,"[그러고, 있, 어, 너무, 무섭, 더, 래, 아빠, 가, 그, 누워있는, 모습, ..."
5057,Sess16_script05_User031M_019,"[어, 확실히]"
7301,Sess26_script05_User051M_008,"[난, 몰랐어, 무한도전, 그, 완전체, 그, 따로, 팬클럽, 있는지]"
...,...,...
2571,Sess09_script05_User017M_015,"[맞아, 내, 몸, 하나, 건, 사하기도, 정말, 힘든, 거, 라는, 거]"
1223,Sess04_script06_User008F_002,"[그거, 에, 대해, 어떻게, 생각, 해]"
8878,Sess32_script01_User064F_010,"[나, 자다가, 이케, 없게, 이케, 하고, 어, 진짜, 숨, 을, 못, 쉬었어, 걔]"
5748,Sess21_script01_User041F_001,"[나, 는, 그거, 있었어]"


In [48]:
import pandas as pd
d = pd.concat([x_train,y_train], axis = 1)
d

,Segment ID,preprocessing,sentiment
2189,Sess07_script06_User014M_007,"[만약, 에, 자식, 이, 라면, 막, 진짜, 로, 막]",0
6259,Sess22_script05_User043F_017,"[그때, 가, 제일, 황당했어, 뭔가, 지금, 까지, 지금, 까지, 생각나는, 황당...",1
10607,Sess39_script03_User078F_016,"[그거, 주소, 도, 적었어]",0
10096,Sess34_script06_User067M_022,"[뽀삐, 를, 내, 가, 10년, 동안, 키웠어]",0
5589,Sess20_script02_User039M_041,"[또, 생각, 하니까, 슬프네]",4
...,...,...,...
6017,Sess21_script06_User041F_024,"[그래, 근데, 그, 황, 씨, 도, 키, 가, 크잖아]",0
709,Sess03_script03_User005M_016,"[그니까, 응, 코, 를, 코, 를, 많이, 고는구나]",0
10679,Sess39_script04_User078F_030,"[어, 막, 날라, 다니는, 자동차, 막, 그런, 것, 들, 도, 굳이, 꼭, 만들...",0
8366,Sess30_script03_User060F_009,"[뭘, 먹, 암만, 맛있는, 김치, 라고, 해서, 사, 와도, 얻어와도, 엄마, 맛...",0


In [49]:
d['sentiment'].value_counts()

0    7900
1     997
2     174
3     146
4     137
5      75
6      43
Name: sentiment, dtype: int64

In [50]:
d = d.reset_index(drop=True)
d = d[['Segment ID', 'sentiment', 'preprocessing']]
d

,Segment ID,sentiment,preprocessing
0,Sess07_script06_User014M_007,0,"[만약, 에, 자식, 이, 라면, 막, 진짜, 로, 막]"
1,Sess22_script05_User043F_017,1,"[그때, 가, 제일, 황당했어, 뭔가, 지금, 까지, 지금, 까지, 생각나는, 황당..."
2,Sess39_script03_User078F_016,0,"[그거, 주소, 도, 적었어]"
3,Sess34_script06_User067M_022,0,"[뽀삐, 를, 내, 가, 10년, 동안, 키웠어]"
4,Sess20_script02_User039M_041,4,"[또, 생각, 하니까, 슬프네]"
...,...,...,...
9467,Sess21_script06_User041F_024,0,"[그래, 근데, 그, 황, 씨, 도, 키, 가, 크잖아]"
9468,Sess03_script03_User005M_016,0,"[그니까, 응, 코, 를, 코, 를, 많이, 고는구나]"
9469,Sess39_script04_User078F_030,0,"[어, 막, 날라, 다니는, 자동차, 막, 그런, 것, 들, 도, 굳이, 꼭, 만들..."
9470,Sess30_script03_User060F_009,0,"[뭘, 먹, 암만, 맛있는, 김치, 라고, 해서, 사, 와도, 얻어와도, 엄마, 맛..."


In [51]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
from konlpy.tag import Okt
okt = Okt()

In [53]:
#각 감정별로 3500개

for i in range(9472):
  if d['sentiment'][i]==1:
    l=EDA(d['preprocessing'][i], alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=3) ####num_aug로 증강 개수 조절
    for k in l:
      j=okt.morphs(k, stem=True)
      d.loc[len(d)]=[d['Segment ID'][i], d['sentiment'][i],j]

  elif d['sentiment'][i]==2:
    l=EDA(d['preprocessing'][i], alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=20) ####num_aug로 증강 개수 조절
    for k in l:
      j=okt.morphs(k, stem=True)
      d.loc[len(d)]=[d['Segment ID'][i], d['sentiment'][i],j]

  elif d['sentiment'][i]==3:
    l=EDA(d['preprocessing'][i], alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=23) ####num_aug로 증강 개수 조절
    for k in l:
      j=okt.morphs(k, stem=True)
      d.loc[len(d)]=[d['Segment ID'][i], d['sentiment'][i],j]

  elif d['sentiment'][i]==4:
    l=EDA(d['preprocessing'][i], alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=25) ####num_aug로 증강 개수 조절
    for k in l:
      j=okt.morphs(k, stem=True)
      d.loc[len(d)]=[d['Segment ID'][i], d['sentiment'][i],j]

  elif d['sentiment'][i]==5:
    l=EDA(d['preprocessing'][i], alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=46) ####num_aug로 증강 개수 조절
    for k in l:
      j=okt.morphs(k, stem=True)
      d.loc[len(d)]=[d['Segment ID'][i], d['sentiment'][i],j]

  elif d['sentiment'][i]==6:
    l=EDA(d['preprocessing'][i], alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=81) ####num_aug로 증강 개수 조절
    for k in l:
      j=okt.morphs(k, stem=True)
      d.loc[len(d)]=[d['Segment ID'][i], d['sentiment'][i],j]
  
d

,Segment ID,sentiment,preprocessing
0,Sess07_script06_User014M_007,0,"[만약, 에, 자식, 이, 라면, 막, 진짜, 로, 막]"
1,Sess22_script05_User043F_017,1,"[그때, 가, 제일, 황당했어, 뭔가, 지금, 까지, 지금, 까지, 생각나는, 황당..."
2,Sess39_script03_User078F_016,0,"[그거, 주소, 도, 적었어]"
3,Sess34_script06_User067M_022,0,"[뽀삐, 를, 내, 가, 10년, 동안, 키웠어]"
4,Sess20_script02_User039M_041,4,"[또, 생각, 하니까, 슬프네]"
...,...,...,...
29654,Sess29_script02_User058F_008,3,"[어, 어떡하다]"
29655,Sess29_script02_User058F_008,3,"[어, 어떡하다]"
29656,Sess25_script05_User050F_010,1,"[난, 요즘, 침, 을, 흘리다, 자가, 지고, 턱, 이, 아, 턱, 이, 아프다]"
29657,Sess25_script05_User050F_010,1,"[난, 요즘, 침, 을, 흘리다, 자가, 지고, 턱, 턱, 이, 아, 턱, 이, 아프다]"


In [54]:
d['sentiment'].value_counts()

0    7900
1    3988
2    3654
4    3562
6    3526
5    3525
3    3504
Name: sentiment, dtype: int64

In [55]:
#sentiment == k 일 때 추출 함수

def drop_senti(d, k, drop_num):
  temp_df = d.loc[d['sentiment']== k, ]
  temp_df = temp_df.sample(n = drop_num)
  drop_list = temp_df.index.tolist()
  d.drop(drop_list, axis=0, inplace=True)
  print("> ---drop 완료---")
  return d

In [56]:
d = drop_senti(d, 0, 4400)
d = drop_senti(d, 1, 488)
d = drop_senti(d, 2, 154)
d = drop_senti(d, 3, 4)
d = drop_senti(d, 4, 62)
d = drop_senti(d, 5, 25)
d = drop_senti(d, 6, 26)

> ---drop 완료---
> ---drop 완료---
> ---drop 완료---
> ---drop 완료---
> ---drop 완료---
> ---drop 완료---
> ---drop 완료---


In [57]:
d['sentiment'].value_counts()

1    3500
0    3500
4    3500
2    3500
3    3500
6    3500
5    3500
Name: sentiment, dtype: int64

In [58]:
x_train = d[['Segment ID', 'preprocessing']]
y_train = d['sentiment']
x_train

,Segment ID,preprocessing
1,Sess22_script05_User043F_017,"[그때, 가, 제일, 황당했어, 뭔가, 지금, 까지, 지금, 까지, 생각나는, 황당..."
2,Sess39_script03_User078F_016,"[그거, 주소, 도, 적었어]"
3,Sess34_script06_User067M_022,"[뽀삐, 를, 내, 가, 10년, 동안, 키웠어]"
4,Sess20_script02_User039M_041,"[또, 생각, 하니까, 슬프네]"
5,Sess25_script02_User049M_013,"[그니까, 뭐, 이제, 뭐, 집, 에, 뭐, 와라, 집들이, 이렇게, 와라, 이런,..."
...,...,...
29653,Sess29_script02_User058F_008,"[어, 어떡하다]"
29654,Sess29_script02_User058F_008,"[어, 어떡하다]"
29655,Sess29_script02_User058F_008,"[어, 어떡하다]"
29656,Sess25_script05_User050F_010,"[난, 요즘, 침, 을, 흘리다, 자가, 지고, 턱, 이, 아, 턱, 이, 아프다]"


In [59]:
tmp_df = pd.concat([x_train,x_valid, test[['Segment ID', 'preprocessing']]], ignore_index=True)
tmp_df

,Segment ID,preprocessing
0,Sess22_script05_User043F_017,"[그때, 가, 제일, 황당했어, 뭔가, 지금, 까지, 지금, 까지, 생각나는, 황당..."
1,Sess39_script03_User078F_016,"[그거, 주소, 도, 적었어]"
2,Sess34_script06_User067M_022,"[뽀삐, 를, 내, 가, 10년, 동안, 키웠어]"
3,Sess20_script02_User039M_041,"[또, 생각, 하니까, 슬프네]"
4,Sess25_script02_User049M_013,"[그니까, 뭐, 이제, 뭐, 집, 에, 뭐, 와라, 집들이, 이렇게, 와라, 이런,..."
...,...,...
28463,Sess40_script06_User079F_043,"[뭔가, 막, 그게, 그니까, 완전, 대중화, 가, 된, 시대, 가, 된, 거야]"
28464,Sess40_script06_User079F_044,"[그, 게임, 속, 그, 스토리, 상, 에서는]"
28465,Sess40_script06_User079F_045,"[근데, 그게, 너무, 저렴하고, 하니까, 다, 갈아, 치우고, 막, 내, 아내, ..."
28466,Sess40_script06_User079F_046,"[왜냐면, 내, 아내, 를, 대신, 할, 사람, 이, 있는, 거야]"


In [60]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tmp_df['preprocessing'])

In [61]:
len(tokenizer.word_index)

12440

In [62]:
#test data 임베딩
x = tokenizer.texts_to_sequences(test['preprocessing'])
test['token']=pd.DataFrame({'token':x})
x_test = test
x_test

,Segment ID,sentiment,preprocessing,token
0,Sess08_script01_User015F_001,1,"[나, 난, 근데, 아까, 봤던, 게, 좀, 그거, 봤던, 것, 들, 이라서]","[9, 77, 20, 508, 1900, 28, 17, 38, 1900, 58, 6..."
1,Sess08_script01_User016F_001,0,"[나, 첫, 번째, 것, 만, 봤었어]","[9, 632, 564, 58, 126, 5849]"
2,Sess08_script01_User015F_002,0,"[근데, 막, 돌아다니는, 거, 많잖아]","[20, 4, 2771, 5, 1996]"
3,Sess08_script01_User016F_002,0,"[맞아, 무한도전, 은]","[250, 613, 50]"
4,Sess08_script01_User015F_003,1,"[그, 맨날, 페북, 이랑, 막, 그런데, 그래가지고, 난, 별로, 재미, 가, 없었어]","[7, 106, 2857, 148, 4, 871, 303, 77, 296, 1213..."
...,...,...,...,...
2609,Sess40_script06_User079F_043,0,"[뭔가, 막, 그게, 그니까, 완전, 대중화, 가, 된, 시대, 가, 된, 거야]","[115, 4, 59, 75, 143, 12438, 29, 751, 746, 29,..."
2610,Sess40_script06_User079F_044,0,"[그, 게임, 속, 그, 스토리, 상, 에서는]","[7, 572, 333, 7, 4603, 494, 466]"
2611,Sess40_script06_User079F_045,0,"[근데, 그게, 너무, 저렴하고, 하니까, 다, 갈아, 치우고, 막, 내, 아내, ...","[20, 59, 32, 12439, 456, 41, 1317, 3694, 4, 22..."
2612,Sess40_script06_User079F_046,0,"[왜냐면, 내, 아내, 를, 대신, 할, 사람, 이, 있는, 거야]","[763, 22, 2453, 26, 2432, 189, 36, 2, 231, 44]"


In [63]:
#train data 임베딩
x_train = x_train.reset_index(drop=True)

x = tokenizer.texts_to_sequences(x_train['preprocessing'])
x_train['token']=pd.DataFrame({'token':x})
x_train

,Segment ID,preprocessing,token
0,Sess22_script05_User043F_017,"[그때, 가, 제일, 황당했어, 뭔가, 지금, 까지, 지금, 까지, 생각나는, 황당...","[55, 29, 137, 4682, 115, 118, 139, 118, 139, 2..."
1,Sess39_script03_User078F_016,"[그거, 주소, 도, 적었어]","[38, 4683, 6, 6358]"
2,Sess34_script06_User067M_022,"[뽀삐, 를, 내, 가, 10년, 동안, 키웠어]","[4684, 26, 22, 29, 1980, 733, 6359]"
3,Sess20_script02_User039M_041,"[또, 생각, 하니까, 슬프네]","[63, 35, 456, 4685]"
4,Sess25_script02_User049M_013,"[그니까, 뭐, 이제, 뭐, 집, 에, 뭐, 와라, 집들이, 이렇게, 와라, 이런,...","[75, 15, 19, 15, 96, 11, 15, 2653, 3828, 39, 2..."
...,...,...,...
24495,Sess29_script02_User058F_008,"[어, 어떡하다]","[8, 436]"
24496,Sess29_script02_User058F_008,"[어, 어떡하다]","[8, 436]"
24497,Sess29_script02_User058F_008,"[어, 어떡하다]","[8, 436]"
24498,Sess25_script05_User050F_010,"[난, 요즘, 침, 을, 흘리다, 자가, 지고, 턱, 이, 아, 턱, 이, 아프다]","[77, 357, 2062, 13, 3771, 1238, 486, 1139, 2, ..."


In [64]:
#valid data 임베딩
x_valid = x_valid.reset_index(drop=True)

x = tokenizer.texts_to_sequences(x_valid['preprocessing'])
x_valid['token']=pd.DataFrame({'token':x})
x_valid

,Segment ID,preprocessing,token
0,Sess16_script01_User031M_007,"[그, 캐, 그, 뭐, 뭐, 냐, 잭스, 패, 로우]","[7, 4537, 7, 15, 15, 545, 9722, 3318, 9723]"
1,Sess34_script03_User068M_031,"[그치, 공항, 에, 연예인, 많지]","[460, 9724, 11, 1723, 1922]"
2,Sess30_script02_User059F_018,"[그러고, 있, 어, 너무, 무섭, 더, 래, 아빠, 가, 그, 누워있는, 모습, ...","[620, 1849, 8, 32, 2675, 64, 1844, 66, 29, 7, ..."
3,Sess16_script05_User031M_019,"[어, 확실히]","[8, 1253]"
4,Sess26_script05_User051M_008,"[난, 몰랐어, 무한도전, 그, 완전체, 그, 따로, 팬클럽, 있는지]","[77, 2879, 613, 7, 9725, 7, 1122, 4107, 2737]"
...,...,...,...
1349,Sess09_script05_User017M_015,"[맞아, 내, 몸, 하나, 건, 사하기도, 정말, 힘든, 거, 라는, 거]","[250, 22, 975, 178, 155, 10706, 396, 1305, 5, ..."
1350,Sess04_script06_User008F_002,"[그거, 에, 대해, 어떻게, 생각, 해]","[38, 11, 393, 324, 35, 54]"
1351,Sess32_script01_User064F_010,"[나, 자다가, 이케, 없게, 이케, 하고, 어, 진짜, 숨, 을, 못, 쉬었어, 걔]","[9, 1225, 1972, 6087, 1972, 122, 8, 12, 426, 1..."
1352,Sess21_script01_User041F_001,"[나, 는, 그거, 있었어]","[9, 40, 38, 636]"


In [65]:
y_test = test['sentiment']

In [66]:
x_test = test['token']
x_train = x_train['token']
x_valid = x_valid['token']

In [67]:
with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/Text_x_train.pkl', 'wb') as f:
    pickle.dump(x_train, f)

with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/Text_y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)

with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/Text_x_valid.pkl', 'wb') as f:
    pickle.dump(x_valid, f)

with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/Text_y_valid.pkl', 'wb') as f:
    pickle.dump(y_valid, f)

with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/Text_x_test.pkl', 'wb') as f:
    pickle.dump(x_test, f)

with open('/content/drive/MyDrive/kcc_휴먼이해논문공모전/Text_y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)